[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10142770.svg)](https://doi.org/10.5281/zenodo.10142770)
# [**SQLxMatch: In-Database Spatial Cross-Match of Astronomical Catalogs**](https://github.com/sciserver/SQLxMatch)

##### Manuchehr Taghizadeh-Popp <sup>1*</sup> and Laszlo Dobos<sup>1,2</sup>
<sup>1</sup> Department of Physics and Astronomy, Johns Hopkins University, Baltimore, MD, USA.<br>
<sup>2</sup> Department of Physics of Complex Systems, Eötvös Loránd University, Budapest, Hungary.<br>
<sup>*</sup> Leading contributor email: mtaghiza [at] jhu.edu  |  Help Desk: sciserver-helpdesk [at] jhu.edu
<br><br>

`SQLxMatch` (or *sequel cross match*)  is a SQL stored procedure that allows to perform 2-dimensional spatial cross-matches and cone searches across multiple astronomical catalogs stored in relational databases.
This procedure implements the `Zones Algorithm` ([[1]](https://arxiv.org/abs/cs/0701171), [[2]](https://arxiv.org/abs/cs/0408031)), which leverages relational database algebra and B-Trees to cross-match the database tables or views containing the catalogs. To run a cross-match, these tables must simply contain at least the Right Ascension (RA) or Longitude, Declination (Dec) or Latitude, and unique object identifier (ID) columns.

We have integrated `SQLxMatch` with more than 50 astronomical catalogs, and made those publicly available as tables in remote SQL Server databases `in the cloud` through the [CasJobs](https://skyserver.sdss.org/CasJobs) website, as part of the [SciServer](https://www.sciserver.org) science platform ([[3]](https://www.sciencedirect.com/science/article/abs/pii/S2213133720300664)). In CasJobs, users can directly execute form-free SQL queries for cross-matching, either synchronously or as asynchronous jobs. For general audiences, a simpler form-based [interactive web interface](http://skyserver.sdss.org/public/CrossMatchTools/crossmatch) that uses the CasJobs REST API for running cross-match queries will be soon available in the [SkyServer](http://skyserver.sdss.org) astronomy portal.<br>
To improve cross-match query execution speed, we install `SQLxMatch` in a SQL Server database supported by fast NVMe storage in a RAID 6 configuration. We also place the catalog tables across several databases in the same physical server, thus avoiding having to move data across servers with a potentially slower network connection.

The advantage of this <i>`in-database`</i> remote cross-match, compared to other <i>`in-memory`</i> local cross-match software libraries, is that the users leverage the remote database server's own (and potentially bigger) computing/memory/storage resources to filter and cross-match the full catalogs right away, having only a relatively small-sized cross-match output table returned to them.
This can be faster and more efficient than having users to download the full catalogs into their own computers (if they have enough storage), and then load them in python for filtering and running the cross-match, for instance.

#### **Web Interface**

[https://skyserver.sdss.org/xmatch](https://skyserver.sdss.org/xmatch)


#### **Citation**

Taghizadeh-Popp, M. and Dobos, L. (2023) “SQLxMatch: In-Database Spatial Cross-Match of Astronomical Catalogs”. Zenodo. doi: 10.5281/zenodo.10142771.

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10142770.svg)](https://doi.org/10.5281/zenodo.10142770)



### The SQLxMatch stored procedure

The first input parameters are the names of 2 catalog tables, views, or temporary tables located in the CasJobs `xmatch` database context. This context already contains several table views from specific astronomical catalogs, and are named with the following format: `CatalogName_TableName`.

The input tables or views are expected to have at least one unique identifier column, as well as an `RA` (Right Ascension) and `Dec` (Declination) columns. <br>
However, the code will run faster if the input tables or views already contain the `cx`, `cy`, and `cz` columns (3-D cartesian coordinates of an object located on the surface of a unit-radius sphere), as well as the `zoneid` column (calculated from a zone height of 

Since the Zones Algorithm internally works with the 3-D cartesian coordinates of an object located on the surface of a unit-radius sphere (named as `cx`, `cy`, and `cz`), the code tries to find those exact names as columns possibly present in the catalog tables. If they are not present, then the code will calculate those cordinates on the fly, based on the values of the `RA` and `Dec` columns.

If the column defining the zone ID (must be named `zoneid`) is missing, then the code will calculate it on the fly, based on the value of the input search radius `@radius` around each object.

The procedure returns a table as `TABLE (id1, id2, sep)`, where `id1` and `id2` are the names of the columns that uniquely identify objects in the first and second catalogs, respectively, and `sep` is the angular separation between objetcs in arseconds.


<b>INPUT PARAMETERS:</b> <br>
<ul>
<li> <b>@table1 NVARCHAR(128) or SYSNAME</b>: name of first input catalog. Can be any of these formats: 'server.database.schema.table', 'database.schema.table', 'database.table', or simply 'table'
<li> <b>@table2 NVARCHAR(128) or SYSNAME</b>: name of first input catalog. Can be any of these formats: 'server.database.schema.table', 'database.schema.table', 'database.table', or simply 'table'
<li> <b>@radius FLOAT</b>: search radius around each object, in arcseconds. Takes a default value of 10 arcseconds.
<li> <b>@id_col1 NVARCHAR(128) or SYSNAME</b>: name of the column defining a unique object identifier in catalog @table1. Takes a default value of 'objid'.
<li> <b>@id_col2 NVARCHAR(128) or SYSNAME</b>: name of the column defining a unique object identifier in catalog @table2. Takes a default value of 'objid'.
<li> <b>@ra_col1 NVARCHAR(128) or SYSNAME</b>: name of the column containing the Right Ascension (RA) in degrees of objects in catalog @table1. Takes a default value of 'ra'.
<li> <b>@ra_col2 NVARCHAR(128) or SYSNAME</b>: name of the column containing the Right Ascension (RA) in degrees of objects in catalog @table2. Takes a default value of 'ra'.
<li> <b>@dec_col1 NVARCHAR(128) or SYSNAME</b>: name of the column containing the Declination (Dec) in degrees of objects in catalog @table1. Takes a default value of 'dec'.
<li> <b>@dec_col2 NVARCHAR(128) or SYSNAME</b>: name of the column containing the Declination (Dec) in degrees of objects in catalog @table2. Takes a default value of 'dec'.
<li> <b>@max_catalog_rows1 BIGINT</b>: default value of NULL. If set, the procedure will use only the TOP @max_catalog_rows1 rows in catalog @table1, with no special ordering.
<li> <b>@max_catalog_rows2 BIGINT</b>: default value of NULL. If set, the procedure will use only the TOP @max_catalog_rows2 rows in catalog @table2, with no special ordering.
<li> <b>@output_table NVARCHAR(128) or SYSNAME</b>: If NOT NULL, this procedure will insert the output results into the table @output_table (of format 'server.database.schema.table', 'database.schema.table', 'database.table', or simply 'table'), which must already exist and be visible within the scope of the procedure. If set to null, the output results will be simply returned as a table resultset. Takes a default value of NULL.
<li> <b>@only_nearest BIT</b>: If set to 0 (default value), then all matches within a distance @radius to an object are returned. If set to 1, only the closest match to an object is returned.
<li> <b>@sort_by_separation BIT</b>: If set to 1, then the output will be sorted by the 'id1' and 'sep' columns. If set to 0 (default value), no particular ordering is applied.
<li> <b>@include_sep_rank BIT</b>: If set to 1, then the output table will include an extra column named 'sep_rank', denoting the rank of all matches to an object when sorted by angular separation. If set to 0 (default value), this column is not included.
<li> <b>@include_radec BIT </b>: If set to 1, then the output table will contain the original (RA, Dec) columns from both input tables, as ra1, dec1, ra2, and dec2.
<li> <b>@print_messages BIT </b>: If set to 1, then time-stamped messages will be printed as the different sections in this procedure are completed.
</ul>

<b>RETURNS:</b> <br>
<ul>
    <li><b>TABLE (id1, id2, sep)</b>, where id1 and id2 are the unique object identifier columns in @table1 and @table2, respectively, and sep (FLOAT) is the angular separation between objects in arseconds. 

<li><b>TABLE (id1, id2, sep, ra1, dec1, ra2, dec2)</b> when @include_radec=1

<li><b>TABLE (id1, id2, sep, sep_rank)</b> when @include_sep_rank=1

<li><b>TABLE (id1, id2, sep, sep_rank, ra1, dec1, ra2, dec2)</b> when @include_radec=1 and @include_sep_rank = 1.
</ul>    


---
# Demo

In order to communicate and send queries to CasJobs, we need to import the CasJobs module from the [SciServer python](https://github.com/sciserver/SciScript-python) package. You will need to install this package if you are not running this notebook in [SciServer-Compute](https://apps.sciserver.org/compute).

In [1]:
from SciServer import CasJobs as cj
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 20)

### Listing all catalogs


Here we run a SQL query containing the `Catalogs` view that lists the names of all available catalogs.<br>

In [2]:
sql = "SELECT * FROM Catalogs ORDER BY catalog_name"
cj.executeQuery(sql, context='xmatch')

,catalog_name,summary,remarks,url
0,ACVS,ASAS Catalog of Variable Stars,\r\n The ASAS-3 Catalog of Variable Stars...,http://www.astrouw.edu.pl/asas/?page=main
1,AGC,Arecibo Galaxy Catalog,"\r\n The AGC, or Arecibo General Catalog,...",http://caborojo.astro.cornell.edu/alfalfalog/i...
2,AKARI,AKARI Point Source Catalogues,\r\n AKARI (Previously known as ASTRO-F o...,http://www.ir.isas.jaxa.jp/AKARI/
3,CHANDRA,"The Chandra Source Catalog, Release 1.1",\r\n The first official release of the CS...,http://cxc.cfa.harvard.edu/csc/
4,CNOC2,The Canadian Network for Observational Cosmol...,\r\n The Canadian Network for Observation...,http://www.astro.utoronto.ca/~cnoc/cnoc2.html
...,...,...,...,...
50,VVDS,The VIMOS VLT Deep Survey,\r\n A total of 11 564 objects have been ...,http://cesam.lam.fr/vvdsproject/index.html
51,WiggleZ,WiggleZ Dark Energy Survey Data Release 1,\r\n The WiggleZ Dark Energy Survey is a ...,http://wigglez.swin.edu.au/site/index.html
52,WISE,\tThe WISE All-Sky data Release,\n NASA's Wide-field Infrared Survey Expl...,http://wise2.ipac.caltech.edu/docs/release/all...
53,WMAP,\tNine-year WMAP point source catalogs,\r\n Nine-year WMAP point source catalogs,https://heasarc.gsfc.nasa.gov/W3Browse/radio-c...


### Listing tables in a catalog


The `Tables` view contains the tables available in all catalogs. We can use the `WHERE` clause to filter rows on the `catalog_name` column.

In [3]:
sql = "SELECT * FROM Tables WHERE catalog_name = 'IRAS'"
cj.executeQuery(sql, context='xmatch')

,catalog_name,table_name,table_type,description,schema_name
0,IRAS,IRAS_PhotoObj,view,The main PhotoObj table for the IRAS catalog,dbo


### Listing columns in a table

Given a table name, we can run this SQL query containing the `getTableColumns` function in order to get a description of all its columns.<br>
This helps identifying the RA and Dec columns, as well as possible column on which we could impose filters.

In [4]:
sql = "SELECT * FROM Columns WHERE table_name = 'IRAS_PhotoObj' ORDER BY column_index"
cj.executeQuery(sql, context='xmatch')

,catalog_name,table_name,column_name,description,unit,ucd,utype,datatype,size,precision,scale,column_index,schema_name
0,IRAS,IRAS_PhotoObj,objID,unique object identifier,,meta.id,,bigint,8,19,0,1,dbo
1,IRAS,IRAS_PhotoObj,name,IRAS Source Name,,meta.id,,varchar,11,0,0,2,dbo
2,IRAS,IRAS_PhotoObj,ra,J2000 right ascension,deg,pos.eq.ra;pos.frame=j2000,,float,8,53,0,3,dbo
3,IRAS,IRAS_PhotoObj,dec,J2000 declination,deg,pos.eq.dec;pos.frame=j2000,,float,8,53,0,4,dbo
4,IRAS,IRAS_PhotoObj,cx,Cartesian coordinate x,,pos.eq.x;pos.frame=j2000,,float,8,53,0,5,dbo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,IRAS,IRAS_PhotoObj,mhcon,"Possible number of HCONs, 99 for NULL",,meta.number,,smallint,2,5,0,55,dbo
55,IRAS,IRAS_PhotoObj,fcor_12,"Flux correction factor applied (times 1000),99...",,stat.param;phot.flux;em.IR.IRAS.12,,int,4,10,0,56,dbo
56,IRAS,IRAS_PhotoObj,fcor_25,"Flux correction factor applied (times 1000),99...",,stat.param;phot.flux;em.IR.IRAS.25,,int,4,10,0,57,dbo
57,IRAS,IRAS_PhotoObj,fcor_60,"Flux correction factor applied (times 1000),99...",,stat.param;phot.flux;em.IR.IRAS.60,,int,4,10,0,58,dbo


---
## Cross-Match Examples

In [5]:
# Setting up seartch parameters. Search radius must be in arcseconds, the rest in degrees.

radius = 30
ra1 = 160
ra2 = 160.5
dec1 = 25
dec2 = 25.5

### Cross-matching two small catalogs - Pandas DataFrame output

Here we cross-match 2 local infrared catalog tables: `IRAS_PhotoObj` and `AKARI_FIS`.

We can retrieve the cross-match output table as a Pandas dataframe by using the syncronous `executeQuery` function in CasJobs, as this cross-match takes less time than its `1 minute timeout`. 

In [6]:
%%time

sql = f"""
EXECUTE SQLxMatch @table1='IRAS_PhotoObj', @table2='AKARI_FIS', @radius=1

-- Note that we could be more explicit by specifying the columns names, but that was not needed.
--EXECUTE SQLxMatch @table1='IRAS_PhotoObj', @id_col1='objid', @ra_col1='ra', @dec_col1='dec',  @table2='AKARI_FIS', @radius={radius}
"""

df = cj.executeQuery(sql, context="xmatch")
print(f"There are {df.shape[0]} matches.")
df

There are 322 matches.
CPU times: user 36.2 ms, sys: 6.06 ms, total: 42.3 ms
Wall time: 9.78 s


,id1,id2,sep
0,95798,3220754,0.824991
1,106298,3224524,0.778418
2,102623,3229378,0.325012
3,11458,3132115,0.749549
4,125607,3251473,0.881630
...,...,...,...
317,74245,3084344,0.603812
318,62100,3169470,0.384570
319,23375,3114499,0.290359
320,137303,3291256,0.909133


### Cross-matching two filtered catalogs - Pandas DataFrame output

Here we use 2 local tables (the `PhotoObjAll` tables in the `BestDR18` and `GalexDR6` catalogs), which we filter by `ra` and `dec` in rectangular regions and store in local temporary tables. These temporary tables are then passed as input to the `SQLxMatch` procedure.

Note that we can also store the (`cx`, `cy`, `cz`) columns in the temporary tables in order to avaoid having the code to compute them later internally on the fly.

Also, note that time we also give `@include_radec`=1, `@include_sep_rank`=1, `@sort_by_separation`=1 as input parameters. 



In [7]:
%%time

sql = f"""

SELECT objid, ra, dec, cx, cy, cz INTO #temp1
FROM SDSSDR18_PhotoObjAll WHERE ra BETWEEN {ra1} AND {ra2} AND dec BETWEEN {dec1} AND {dec2}

SELECT objid, ra, dec, cx, cy, cz INTO #temp2 
FROM GalexGR6_PhotoObjAll WHERE ra BETWEEN {ra1} AND {ra2} AND dec BETWEEN {dec1} AND {dec2}

EXECUTE SQLxMatch @table1='#temp1', @table2='#temp2', @radius={radius}, @include_radec=1, @include_sep_rank=1, @sort_by_separation=1
"""

df = cj.executeQuery(sql, context="xmatch")
print(f"There are {df.shape[0]} matches.")
df

There are 14482 matches.
CPU times: user 175 ms, sys: 59.5 ms, total: 234 ms
Wall time: 4.66 s


,id1,id2,sep,sep_rank,ra1,dec1,ra2,dec2
0,1237667322710261954,6387874658268479704,1.798951,1,160.008407,25.004219,160.008487,25.003724
1,1237667322710261954,6387874659342225269,12.009139,2,160.008407,25.004219,160.011864,25.003073
2,1237667322710261955,6387874658268479704,1.792355,1,160.008402,25.004216,160.008487,25.003724
3,1237667322710261955,6387874659342225269,12.022925,2,160.008402,25.004216,160.011864,25.003073
4,1237667322710261983,6387874658268479748,23.739031,1,160.027793,25.022443,160.034873,25.020916
...,...,...,...,...,...,...,...,...
14477,1237667430638814217,6387874658270578860,27.791207,1,160.041967,25.474943,160.033920,25.477554
14478,1237667430638814217,6387874658269533411,29.379765,2,160.041967,25.474943,160.042697,25.466809
14479,1237667430638814224,6387874658268481825,23.994318,1,160.036649,25.495353,160.029969,25.498194
14480,1237667430638814224,6387874658270578846,26.040727,2,160.036649,25.495353,160.037995,25.488222


### Cross-Match beween 2 catalogs as an asynchronous job - Output to MyDB

When the cross-match is expected to take longer than the 1 minute timeout, it is recommendable to run it as an asynchronous job store the cross-match results asynchronous job into a table in `MyDB`.

First, one must create the output table in MyDB, or delete if this table was already created by this demo.


In [8]:
mydb_output_table_name = "xmatch_table"

sql = f"""
IF EXISTS (select * from sys.objects WHERE object_id = OBJECT_ID(N'{mydb_output_table_name}') AND TYPE = 'U')
DROP TABLE {mydb_output_table_name}  
CREATE TABLE {mydb_output_table_name}(id1 BIGINT, id2 BIGINT, sep FLOAT, sep_rank INTEGER, ra1 BIGINT, dec1 FLOAT, ra2 FLOAT, dec2 FLOAT)
"""
df = cj.executeQuery(sql, context="mydb")

<br>
The result of the `sp_xmatch` procedure can be stored in a local temporary table specified by the `@output_table` input parameter. This output table can be then used to fill the table in MyDB.
<br>

In [9]:
%%time

sql = f"""
SELECT objid, ra, dec INTO #temp1
FROM SDSSDR18_PhotoObjAll WHERE ra BETWEEN {ra1} AND {ra2} AND dec BETWEEN {dec1} AND {dec2}

SELECT objid, ra, dec into #temp2 
FROM GALEXGR6_PhotoObjAll WHERE ra BETWEEN {ra1} AND {ra2} AND dec BETWEEN {dec1} AND {dec2}

-- Creating temporary output table
CREATE TABLE #out(id1 BIGINT, id2 BIGINT, sep FLOAT, sep_rank INTEGER, ra1 BIGINT, dec1 FLOAT, ra2 FLOAT, dec2 FLOAT)

-- Executing cross-match that fills output table.
EXECUTE SQLxMatch @table1='#temp1', @table2='#temp2', @radius={radius}, @output_table='#out', @include_radec=1, @include_sep_rank=1

-- Filling up table in MyDB:
INSERT INTO mydb.{mydb_output_table_name}
SELECT * from #out
"""

job_id = cj.submitJob(sql, context="xmatch")

# this line will make the code wait until the job is done, if desired:
job_description = cj.waitForJob(job_id)
print(f"Job Message: {job_description.get('Message')}")

Job Message: Query Complete
CPU times: user 143 ms, sys: 11.6 ms, total: 154 ms
Wall time: 17.2 s


<br>
We can now inspect the contents of the MyDB output table:

In [10]:
sql = f"""
SELECT * FROM mydb.{mydb_output_table_name} ORDER BY id1, sep
"""
df = cj.executeQuery(sql, context="mydb")
print(f"There are {df.shape[0]} matches.")
df

There are 14482 matches.


,id1,id2,sep,sep_rank,ra1,dec1,ra2,dec2
0,1237667322710261954,6387874658268479704,1.798951,1,160,25.004219,160.008487,25.003724
1,1237667322710261954,6387874659342225269,12.009139,2,160,25.004219,160.011864,25.003073
2,1237667322710261955,6387874658268479704,1.792355,1,160,25.004216,160.008487,25.003724
3,1237667322710261955,6387874659342225269,12.022925,2,160,25.004216,160.011864,25.003073
4,1237667322710261983,6387874658268479748,23.739031,1,160,25.022443,160.034873,25.020916
...,...,...,...,...,...,...,...,...
14477,1237667430638814217,6387874658270578860,27.791207,1,160,25.474943,160.033920,25.477554
14478,1237667430638814217,6387874658269533411,29.379765,2,160,25.474943,160.042697,25.466809
14479,1237667430638814224,6387874658268481825,23.994318,1,160,25.495353,160.029969,25.498194
14480,1237667430638814224,6387874658270578846,26.040727,2,160,25.495353,160.037995,25.488222
